##### 1. Non-dominated Sorting Genetic Algorithm (NSGA-II)
##### 2. Multi-Objective Particle Swarm Optimization (MOPSO)
##### 3. Multi-Objective Differential Evolution (MODE)
### 4. Multi-Objective Genetic Algorithm (MOGA)

In [26]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from model.networks import get_all_devices_combined
from model.services import get_all_services

# Define the problem as a multi-objective optimization problem
creator.create("FitnessMulti", base.Fitness, weights=[-1.0, -1.0])
creator.create("Individual", list, fitness=creator.FitnessMulti)

physical_machines = get_all_devices_combined("config-1")
services = get_all_services("config-1")

# Define the problem-specific parameters
n = len(physical_machines)  # Number of physical machines
num_generations = 100
s = len(services)  # Number of services
# Define the objective functions with alpha and beta
alpha = 0.5  # Weight for latency
beta = 0.5   # Weight for energy consumption

# Define parameters. This will be loaded from the database.
# Here, we use random values for demonstration purposes.
# Replace these with actual data when implementing in a real system.

# Variable number of virtual machines for each physical machine
max_v = max(map(lambda machine: len(machine.guest_machines), physical_machines))

################ Physical and virtual machine configurations ##################

# Initialization
R = np.empty(shape=(n, max_v))  # Request network delay of the machine
R.fill(999999) # Initialize with very large value
S = np.empty(shape=(n, max_v))  # Response network delay of the machine
S.fill(999999) # Initialize with very large value
I = np.empty(shape=(n, max_v))  # IPS currently executed
I.fill(0) # Initialize with 0
X = np.empty(shape=(n, max_v))  # Maximum IPS possible
X.fill(1) # Initialize with 1

PI = np.empty(shape=(n, max_v))  # Power consumption in idle
PI.fill(999) # Initialize with very large value
PM = np.empty(shape=(n, max_v))  # Maximum power consumption
PM.fill(999) # Initialize with very large value

# Fill in with the actual numbers
for i in range(len(physical_machines)):
    for j in range(len(physical_machines[i].guest_machines)):
        R[i][j] = physical_machines[i].guest_machines[j].net_delay_request
        S[i][j] = physical_machines[i].guest_machines[j].net_delay_response
        X[i][j] = physical_machines[i].guest_machines[j].max_instructions_per_second
        PI[i][j] = physical_machines[i].guest_machines[j].idle_cpu_utilization
        PM[i][j] = physical_machines[i].guest_machines[j].max_cpu_utilization

################ Service configurations ###################

# Acceptable latencies for services
SAL = [service.acceptable_latency for service in services]
# Initialize IPS values for services (replace with actual data)
SIPS = [service.average_instructions_per_second for service in services]
# Layers considered for service
SL = [service.layer for service in services]

# Define the objective functions
def evaluate_individual(individual):
    # Calculate energy consumption based on the parameters
    E = PI + (PM - PI) * (I / X)
    total_latency = 0.0
    total_energy = 0.0
    success_count = 0
    for i in range(s):
        p = individual[i]  # Physical machine where service is placed
        if p < n:
            num_virtual_machines = len(physical_machines[p].guest_machines)
            for v in range(len(physical_machines[p].guest_machines)):
                
                vm = physical_machines[p].guest_machines[v]
                latency = R[p][v] + (SIPS[i] / X[p][v]) + S[p][v]
                if SL[i] is not None and SL[i] != vm.layer: # Consider the layer constrains provided by the service
                    continue
                if latency <= SAL[i]:
                    total_latency += latency
                    current_instructions = PI[p][v] + SIPS[i]
                    E[p][v] = PI[p][v] + (PM[p][v] - current_instructions) * (I[p][v] / X[p][v]) # Update current energy consumption
                    total_energy += E[p][v]  # Calculate energy consumption
                    success_count += 1
    
    # if all the services are not placed for this particular individual,
    # return a large fitness value to ignore it
    if success_count is not len(services):
        total_latency = 1000
        total_energy = 1000
        
    # Calculate the weighted sum of objectives
    weighted_latency = alpha * total_latency
    weighted_energy = beta * total_energy
    return np.array([weighted_latency, weighted_energy])

# Create a DEAP toolbox and register functions
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, n - 1)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int,), n=s)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=n - 1, indpb=0.2)
toolbox.register("select", tools.selBest)

# Crossover, mutation, and selection registration

# In the main loop, consider both latency and energy in the fitness function

if __name__ == "__main__":
    # Create the initial population
    population = toolbox.population(n=100)

    # Create statistics object to track performance
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min_lat", min)
    stats.register("min_energy", min)

    # Create a logbook to log statistics
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min_lat", "min_energy"

    # Run the optimization using a basic Genetic Algorithm (GA)
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, stats=stats,
                        halloffame=None, verbose=True)

    # Extract the Pareto front solutions (not necessarily Pareto-optimal in a basic GA)
    pareto_front = population

    # Print Pareto front solutions
    print("Pareto Front Solutions:")
    for ind in pareto_front:
        print("Latency:", ind.fitness.values[0])  # Extract the first element
        print("Energy Consumption:", ind.fitness.values[1])  # Extract the second element
        print("Placement:", ind)

gen	nevals	min_lat       	min_energy    
0  	100   	(500.0, 500.0)	(500.0, 500.0)
1  	86    	(116.14478335418258, 335.7162690514614)	(116.14478335418258, 335.7162690514614)
2  	67    	(116.14478335418258, 335.7162690514614)	(116.14478335418258, 335.7162690514614)
3  	71    	(138.38637539831734, 359.3303570290544)	(138.38637539831734, 359.3303570290544)
4  	84    	(500.0, 500.0)                         	(500.0, 500.0)                         
5  	78    	(500.0, 500.0)                         	(500.0, 500.0)                         
6  	78    	(500.0, 500.0)                         	(500.0, 500.0)                         
7  	81    	(500.0, 500.0)                         	(500.0, 500.0)                         
8  	75    	(500.0, 500.0)                         	(500.0, 500.0)                         
9  	70    	(500.0, 500.0)                         	(500.0, 500.0)                         
10 	62    	(500.0, 500.0)                         	(500.0, 500.0)                         
11 	82  